In [1]:
import pandas as pd
import numpy as np
import sqlite3

# --- 1. Generar Datos Simulados ---
print("Generando datos de sensores...")

# Definimos los parámetros
num_sensores_temp = 3
num_sensores_pres = 2
num_muestras = 1000 # Número de momentos en el tiempo

# Creamos un rango de fechas
timestamps = pd.to_datetime(pd.date_range(start='2025-01-01', periods=num_muestras, freq='min'))

# Diccionario para guardar los datos
datos = {'timestamp': [], 'sensor_id': [], 'valor': []}

# Generamos lecturas "normales" para cada sensor
np.random.seed(42) # Para que los resultados sean reproducibles
for ts in timestamps:
    # Sensores de temperatura
    for i in range(num_sensores_temp):
        datos['timestamp'].append(ts)
        datos['sensor_id'].append(f'temp_{i+1}')
        datos['valor'].append(np.random.normal(loc=150, scale=5)) # Normal alrededor de 150°C
    
    # Sensores de presión
    for i in range(num_sensores_pres):
        datos['timestamp'].append(ts)
        datos['sensor_id'].append(f'pres_{i+1}')
        datos['valor'].append(np.random.normal(loc=30, scale=1.5)) # Normal alrededor de 30 PSI

df_largo = pd.DataFrame(datos)


# --- 2. Crear Base de Datos y Cargar Datos ---
db_file_path = 'planta_quimica.db'
print(f"Creando la base de datos '{db_file_path}'...")

conn = sqlite3.connect(db_file_path)
df_largo.to_sql('lecturas_sensores', conn, if_exists='replace', index=False)
conn.close()

print("¡Éxito! La base de datos y la tabla 'lecturas_sensores' han sido creadas.")

db_file_path = 'planta_quimica.db'

# 1. Conectarse a la base de datos
conn = sqlite3.connect(db_file_path)

# 2. Definir la consulta SQL para "pivotar" los datos
query = """
    SELECT
        timestamp,
        MAX(CASE WHEN sensor_id = 'temp_1' THEN valor END) AS temp_1,
        MAX(CASE WHEN sensor_id = 'temp_2' THEN valor END) AS temp_2,
        MAX(CASE WHEN sensor_id = 'temp_3' THEN valor END) AS temp_3,
        MAX(CASE WHEN sensor_id = 'pres_1' THEN valor END) AS pres_1,
        MAX(CASE WHEN sensor_id = 'pres_2' THEN valor END) AS pres_2
    FROM
        lecturas_sensores
    GROUP BY
        timestamp;
"""

# 3. Ejecutar la consulta y cargar los resultados en un DataFrame
print("Ejecutando la consulta para pivotar la tabla...")
df_ancho = pd.read_sql_query(query, conn)

# 4. Cerrar la conexión
conn.close()

# 5. Mostrar las primeras 5 filas para verificar el resultado
print("\n¡Tabla pivotada exitosamente! Estas son las primeras 5 filas:")
print(df_ancho.head())

Generando datos de sensores...
Creando la base de datos 'planta_quimica.db'...
¡Éxito! La base de datos y la tabla 'lecturas_sensores' han sido creadas.
Ejecutando la consulta para pivotar la tabla...

¡Tabla pivotada exitosamente! Estas son las primeras 5 filas:
             timestamp      temp_1      temp_2      temp_3     pres_1  \
0  2025-01-01 00:00:00  152.483571  149.308678  153.238443  32.284545   
1  2025-01-01 00:01:00  148.829315  157.896064  153.837174  29.295788   
2  2025-01-01 00:02:00  147.682912  147.671351  151.209811  27.130080   
3  2025-01-01 00:03:00  147.188562  144.935844  151.571237  28.637964   
4  2025-01-01 00:04:00  157.328244  148.871118  150.337641  27.862878   

      pres_2  
0  29.648770  
1  30.813840  
2  27.412623  
3  27.881544  
4  29.183426  
